    %load_ext pidgin.shell
    
The `pidgin` shell augments the `IPython` shell in the notebook to permit Markdown forward [computational essays](http://blog.stephenwolfram.com/2017/11/what-is-a-computational-essay/).   It provides:

* Markdown input in code cells.
* Markdown block code statements and expressions are evaluated as a normal code cell.
* Inline code expressions are evaluated asynchronously.
* The Markdown code accepts `jinja2` templates too


    IPython.core.interactiveshell.InteractiveShell._run_cell??

In [1]:
    import IPython, traitlets, jinja2, types, contextlib, textwrap, ast, argparse, traceback, doctest
    try:
        from .tangle import markdown_to_python_user_expressions
    except:
        from tangle import markdown_to_python_user_expressions

In [2]:
    def compare_source(raw: str, source: str) -> bool: return list(
        map(str.rstrip, textwrap.dedent(raw).splitlines())
    ) == list(map(str.rstrip, textwrap.dedent(source).splitlines()))

In [3]:
    class DocTestException(BaseException):
        def _render_traceback_(self):
            print('butts')
            return args[0].splitlines()

In [4]:
    class InlineCodeException(BaseException): ...

In [5]:
    def show_failures(str):
        IPython.display.display(IPython.display.Pretty(str))

In [6]:
    import doctest

In [7]:
    class DocTest:
        exception = None
        def _raise_exc(self, str):
            self.exception = DocTestException(str)
        def __call__(self, ip, str, examples):
            import doctest
            doctest.DocTestRunner().run(
                 doctest.DocTest(
                    examples=examples,
                    globs=ip.user_ns, 
                    name='__main__', 
                    filename='__main__', 
                    lineno=0, 
                    docstring=str
                 ), out=self._raise_exc)
            return self.exception

    doctest.DebugRunner??

In [8]:
    import sys

In [9]:
    class PidginShell(IPython.core.interactiveshell.InteractiveShell):
        def run_cell(self, raw, store_history=False, silent=False, shell_futures=True, **user_expressions):
            """Run cell separates the sync and async parts."""
            self._last_traceback = None
            
            source, user_expressions, examples = markdown_to_python_user_expressions(raw)
            
            result = IPython.core.interactiveshell.InteractiveShell.run_cell(
                self, source, store_history=store_history, 
                silent=silent, shell_futures=shell_futures)
            
            if not (result.error_in_exec or result.error_before_exec):
                if examples:
                    doctests = doctest.DocTestRunner().run(
                         doctest.DocTest(
                            examples=examples,
                            globs=self.user_ns, 
                            name='__main__', 
                            filename='__main__', 
                            lineno=0, 
                            docstring=str
                         ), out=show_failures)
                    if doctests.failed: return result
                    

                is_source =  compare_source(raw, source)
                try:
                    is_display = bool(raw.partition('\n')[0].strip() and not raw.startswith(';')) 
                    if raw.strip() == ast.literal_eval(source).strip():
                        is_display = False
                except: ...

                self.user_expressions({str: str for str in user_expressions})
                
                if is_display and not is_source: IPython.display.display(raw)

                
            return result
        
        def user_expressions(self, expressions):
            """User expressions evaluate expressions, assert statements are the only exception. `assert` may
            augment a literate narrative so we choose to include these statements in the expressions.
            """
            out = {}
            user_ns = self.user_ns
            global_ns = self.user_global_ns
            value = {}
            for key, expression in expressions.items():
                expression = self.input_transformer_manager.transform_cell(expression)
                try:
                    evaluate = (key.startswith('assert ') and exec or eval) # assertions exec, everything else evals.
                    value = self._format_user_obj(
                        evaluate(expression, global_ns, user_ns)
                    )
                except:
                    self._user_obj_error()
                    value = None
                    break
            if value is None:
                IPython.display.display(IPython.display.Markdown('---'))
                traceback.print_last()
            return out        

In [10]:
    import traceback

In [11]:
    class PidginShell(IPython.core.interactiveshell.InteractiveShell):
        def run_cell(self, raw, store_history=False, silent=False, shell_futures=True, **user_expressions):
            """Run cell separates the sync and async parts."""
            self._last_traceback = None
            
            source, user_expressions, examples = markdown_to_python_user_expressions(raw)
            
            result = IPython.core.interactiveshell.InteractiveShell.run_cell(
                self, source, store_history=store_history, 
                silent=silent, shell_futures=shell_futures)
            
            if not (result.error_in_exec or result.error_before_exec):
                doctests = DocTest()
                test_results = result.error_in_exec = doctests(self, str, examples)
                if test_results: 
                    self.showtraceback((type(test_results), test_results, test_results.__traceback__))
                        
            if not (result.error_in_exec or result.error_before_exec):
                is_source =  compare_source(raw, source)
                try:
                    is_display = bool(raw.partition('\n')[0].strip() and not raw.startswith(';')) 
                    if raw.strip() == ast.literal_eval(source).strip():
                        is_display = False
                except: ...

                try:
                    self.user_expressions({str: str for str in user_expressions})
                    if is_display and not is_source: IPython.display.display(raw)
                except BaseException as Exception:
    #                     Exception = Exception.with_traceback(BaseException().__traceback__)
                    result.error_in_exec = Exception
                    self.showtraceback((type(Exception), Exception, Exception.__traceback__), tb_offset=3)
            return result
        
        def user_expressions(self, expressions):
            """User expressions evaluate expressions, assert statements are the only exception. `assert` may
            augment a literate narrative so we choose to include these statements in the expressions.
            """
            out = {}
            user_ns = self.user_ns
            global_ns = self.user_global_ns
            value = {}
            for key, expression in expressions.items():
                expression = self.input_transformer_manager.transform_cell(expression)
                evaluate = (key.startswith('assert ') and exec or eval) # assertions exec, everything else evals.
                try:
                    value = self._format_user_obj(evaluate(expression, global_ns, user_ns))
                except BaseException as Exception:
                    raise InlineCodeException(expression) from Exception
            return out

In [12]:
    original_methods = {}
    def load_ipython_extension(ip):
        ip.run_cell = types.MethodType(PidginShell.run_cell, ip)
        ip.user_expressions = types.MethodType(PidginShell.user_expressions, ip)
        ip.enable_html_pager = True
    def unload_ipython_extension(ip):  
        ip.run_cell = types.MethodType(type(ip).run_cell, ip)

In [13]:
    parser = argparse.ArgumentParser(description="""Modify settings for a running pidgin kernel.""")

In [14]:
    if __name__ == '__main__': load_ipython_extension(IPython.get_ipython())